In [1]:
import intake
from deepdiff import DeepDiff
import yaml
import pystac
import warnings
warnings.filterwarnings('ignore')
import intake_stac
intake_stac.__version__

'0.3.0.post66'

In [2]:
item_url = str('intake_stac/tests/data/1.0.0/collection/simple-item.json')
pystac_item = pystac.Item.from_file(item_url)

cat1 = intake_stac.StacItem(pystac_item)
cat1_str = cat1.yaml() #This is the YAML method we implemented

d = yaml.load(cat1_str)

In [3]:
# Test Cat Item YAML
for key in ['bbox','date','datetime','geometry','version']:
    assert key in d['metadata']
for key in ['B02','B03']:
    assert key in d['sources']

In [4]:
#Test Cat Item YAML Roundtrip
test_file = 'test.yaml'
with open(test_file, 'w') as f:
    f.write(cat1_str)

cat2 = intake.open_catalog(test_file)

for key in ['B02','B03']:
    assert key in cat2
        
#assert cat1.walk() == cat2.walk()

In [5]:
key_diff = set(cat2.walk()) - set(cat1.walk())
print(key_diff)

value_diff = { k : cat2.walk()[k] for k in set(cat2.walk()) - set(cat1.walk()) }
print(value_diff)

cat2.walk() == cat1.walk()

set()
{}


False

In [6]:
diff = DeepDiff(cat1.walk(), cat2.walk())
diff

{'type_changes': {"root['B02']": {'old_type': intake_stac.catalog.StacAsset,
   'new_type': intake.catalog.local.LocalCatalogEntry,
   'old_value': name: B02
   container: xarray
   plugin: ['rasterio']
   driver: ['rasterio']
   description: Band 2 - Blue
   direct_access: allow
   user_parameters: []
   metadata: 
     href: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/17/Q/LA/2017/12/27/S2B_MSIL2A_20171227T160459_N0212_R054_T17QLA_20201014T165101.SAFE/GRANULE/L2A_T17QLA_A004227_20171227T160750/IMG_DATA/R10m/T17QLA_20171227T160459_B02_10m.tif
     type: image/tiff; application=geotiff; profile=cloud-optimized
     title: Band 2 - Blue
     eo:bands: [{'name': 'B02', 'common_name': 'blue', 'description': 'Band 2 - Blue', 'center_wavelength': 0.49, 'full_width_half_max': 0.098}]
     gsd: 10
     proj:shape: [10980, 10980]
     proj:bbox: [300000, 1990200, 409800, 2100000]
     proj:transform: [10, 0, 300000, 0, -10, 2100000]
     roles: ['data']
     plots: 
       geotif

In [10]:
type(cat1['B02'])

intake_xarray.raster.RasterIOSource

In [11]:
type(cat2['B02'])

intake_xarray.raster.RasterIOSource